In [25]:
import pandas as pd
import numpy as np
import math
import json
import requests

In [2]:
df = pd.read_csv('../resources/ukb_synthetic_data_tofu.csv')
df

,eid,Sample name,Accession,Subject id,EGA sample id,Platelet count-0.0,Ethnic background-0.0,Body mass index (BMI)-0.0,Weight-0.0,Diabetes diagnosed by doctor-0.0,...,Operative procedures - OPCS4-0.0,Frequency of drinking alcohol-0.0,"Blood, blood-forming organs and certain immune disorders","Endocrine, nutritional and metabolic diseases",Mental and behavioural disorders,Nervous system disorders,Digestive system disorders,Respiratory system disorders,Circulatory system disorders,Covid-19 tests results
0,fake1,UKB_FAKE_1,SAMEA7082814,HG00096,EGAN00002587077,244.9589,African,26.638383,85.6358,Yes,...,L46.3 Ligation of visceral branch of abdominal...,Monthly or less,NaN,NaN,NaN,alzheimer's disease,NaN,NaN,NaN,Negative
1,fake2,UKB_FAKE_2,SAMEA7083131,HG00097,EGAN00002588943,215.7736,British,25.622784,64.5722,Yes,...,K06.1 Repositioning of transposed great arteries,4 or more times a week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
2,fake3,UKB_FAKE_3,SAMEA7082831,HG00099,EGAN00002589008,312.3504,Asian or Asian British,22.359763,64.4364,Prefer not to answer,...,T89.4 Cannulation of lymphatic duct,2 to 3 times a week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
3,fake4,UKB_FAKE_4,SAMEA7084191,HG00100,EGAN00002587535,150.0247,White and Asian,28.278855,74.4885,Prefer not to answer,...,T77.2 Wide excision of muscle,2 to 3 times a week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
4,fake5,UKB_FAKE_5,SAMEA7082531,HG00101,EGAN00002589004,215.5871,White and Black African,25.520670,82.3472,Yes,...,Z01.4 Tissue of occipital lobe of brain,Never,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,fake2517,UKB_FAKE_2517,SAMEA7784064,NA11993,NaN,217.7275,Pakistani,34.258400,44.8799,Yes,...,B09.2 Excision of sublingual thyroid tissue,2 to 4 times a month,sarcoidosis,NaN,bipolar affective disorder,huntington's disease,NaN,NaN,NaN,Negative
2517,fake2518,UKB_FAKE_2518,SAMEA7784065,NA18487,NaN,252.7306,Bangladeshi,26.639700,80.7412,Prefer not to answer,...,O17.9 Unspecified secondary closed reduction o...,2 to 4 times a month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
2518,fake2519,UKB_FAKE_2519,SAMEA7784066,NA19150,NaN,175.0045,Any other Asian background,37.562400,82.2092,Yes,...,D19.8 Other specified extirpation of lesion of...,Monthly or less,NaN,NaN,NaN,NaN,fibrosis and cirrhosis of liver,NaN,NaN,Negative
2519,fake2520,UKB_FAKE_2520,SAMEA7784067,NA20537,NaN,310.7993,British,27.870900,76.0864,Do not know,...,Z22.9 Nose NEC,Prefer not to answer,NaN,NaN,NaN,NaN,dental caries,NaN,cardiomyopathy,Negative


In [24]:
# disorders = [x for x in df['Nervous system disorders']]
nervous_disorders = df['Nervous system disorders'].unique()
nervous_disorders = [ x for x in nervous_disorders if x == x]
print(nervous_disorders)

alzheimer's disease
multiple sclerosis
huntington's disease


In [26]:
annotator_endpoint = 'http://45.88.81.24:8080/annotate'
def annotate(term, model, concept):
  params = {"text": term, "model": model, "concept": concept}
  response = requests.get(annotator_endpoint, params=params)
  return response.json()

annotate("alzheimer's disease", "ZOOMA", "disease")


{'text': "alzheimer's disease",
 'annotations': [{'text': "alzheimer's disease",
   'ontology': {'id': 'MONDO:0004975', 'label': 'Alzheimer disease'},
   'score': 80,
   'source': 'Zooma'}]}

In [ ]:
annotations = []
for x in nervous_disorders:
    zooma_annotation = annotate(x, "ZOOMA", "disease")
    if zooma_annotation["annotations"]:
        ontology = zooma_annotation["annotations"][0]["ontology"]
        zooma_annotation =  ontology["id"] + " - " + ontology["label"]
    else:
        zooma_annotation = "-"

    hesso_annotation = annotate(x, "HESSO_SIB", "disease")
    if hesso_annotation["annotations"]:
        ontology = hesso_annotation["annotations"][0]["ontology"]
        hesso_annotation =  ontology["id"] + " - " + ontology["label"]
    else:
        hesso_annotation = "-"

    annotations.append({
        "term": x,
        "zooma": zooma_annotation,
        "hesso_sib": hesso_annotation
    })

df = pd.DataFrame(annotations)
df.head()
